In [7]:
search_for = 83
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004384756088256836
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 48037081
type: [1, 1, 1, 1, 83] 83
cases of this type: 47458321
10000 6.9749014976783
20000 7.418537301376919
30000 27.063825097864697
40000 25.11686988227451
50000 26.0493390210064
60000 31.972895032288744
70000 28.582310291642607
80000 32.30630634634391
90000 31.136537616939957
100000 23.98672164356669
110000 29.32962473741555
120000 30.919650176326186
130000 26.146331806599143
140000 32.36879051000889
150000 30.192591706248926
160000 26.430928501473904
170000 26.975690776378993
180000 24.678848021133103
190000 23.51375899066244
200000 26.01762428237774
210000 24.778808877800383
220000 24.63584175437951
230000 27.79742402153545
240000 24.10343318839434
250000 23.803065133293686
260000 25.16504604611468
270000 27.426923464714044
280000 4.1442847704655925
290000 4.173023064879649
300000 4.162719900352332
310000 22.029403537633836

3060000 20.087344631019498
3070000 18.69434086423
3080000 20.409511519870062
3090000 20.692962096508662
3100000 20.74942889160669
3110000 18.714791574741085
3120000 18.918557681937838
3130000 17.07244041950846
3140000 3.9475418158199784
3150000 3.854010157770342
3160000 3.937062619703968
3170000 18.335307387828465
3180000 18.668971473548012
3190000 18.538757158857283
3200000 19.490865248186385
3210000 19.54599237549531
3220000 19.62834533684393
3230000 19.31035338621968
3240000 19.631409584273914
3250000 18.867293127709093
3260000 19.502892536440424
3270000 21.1151093578769
3280000 21.167058245548066
3290000 19.98230710981497
3300000 19.94489079436315
3310000 20.07423912830021
3320000 18.761391089553012
3330000 19.780643259170695
3340000 19.428257206100483
3350000 19.844262560464298
3360000 20.739144789702898
3370000 20.30327293248421
3380000 16.24994219281341
3390000 9.276371768248486
3400000 9.211706635812025
3410000 9.132676682851917
3420000 24.04356067757499
3430000 21.822914127367

6150000 18.857106480355228
6160000 20.011633491585194
6170000 19.019678909849258
6180000 18.832876165731395
6190000 18.668783631579437
6200000 19.55822044710502
6210000 8.348353005022943
6220000 8.298241183094262
6230000 8.459885749215497
6240000 16.630515919852048
6250000 20.203509188937662
6260000 19.01370022704227
6270000 18.073562442141466
6280000 20.052386598963956
6290000 20.013364794377644
6300000 21.30665493580253
6310000 20.181133140672387
6320000 19.86727493044765
6330000 19.571548406822085
6340000 18.897507688397585
6350000 21.781916917091536
6360000 9.5463877032385
6370000 7.055458715370178
6380000 6.818561115591023
6390000 12.688925749254967
6400000 21.528673664468506
6410000 19.24557559863908
6420000 19.841977337233697
6430000 20.935407123519482
6440000 18.506736495561594
6450000 19.70098183051212
6460000 18.21806071058786
6470000 19.96225070933532
6480000 19.553578101169606
6490000 18.99226477000425
6500000 17.295257161896803
6510000 17.55345785749634
6520000 16.33861136

9230000 17.26723527829499
9240000 17.802302684645188
9250000 13.1666189267933
9260000 6.6285725296994515
9270000 6.263606750498149
9280000 7.572894132036627
9290000 18.416462994541597
9300000 16.470067066413474
9310000 16.936478975613653
9320000 15.878021705776579
9330000 16.68622262611098
9340000 14.22741285227281
9350000 16.617921578122992
9360000 14.44471721366216
9370000 16.22503023437619
9380000 14.177984529205622
9390000 16.24650609463609
9400000 14.988186359333396
9410000 14.305618365249142
9420000 14.722642081814294
9430000 3.40831731878521
9440000 3.324771351383911
9450000 3.319341696468075
9460000 15.689954489860334
9470000 14.699540530354838
9480000 15.067133051943307
9490000 14.948205387838005
9500000 13.563058907244795
9510000 13.39814739519704
9520000 14.348755305937768
9530000 16.607108084594767
9540000 15.771353183863527
9550000 15.958996044917503
9560000 15.821969380096338
9570000 17.559321925356766
9580000 14.678678872477233
9590000 18.809707215962714
9600000 7.647402

12230000 16.327587269560443
12240000 13.730486346339166
12250000 15.714651696796794
12260000 11.740898381850526
12270000 15.324188459874165
12280000 10.389206864937186
12290000 2.8614597835977076
12300000 2.827708144568364
12310000 2.8568543483684463
12320000 16.296546422143905
12330000 13.76217768027782
12340000 15.657346493653913
12350000 13.580679219104708
12360000 15.205737523081739
12370000 13.21478254178506
12380000 13.082823191881324
12390000 12.701950759607772
12400000 14.061516569756765
12410000 15.969053903583927
12420000 10.175961974920842
12430000 6.640546796622674
12440000 6.537548330727908
12450000 9.396300678448869
12460000 15.68328270934379
12470000 15.784685588420892
12480000 14.839968433657202
12490000 15.050230754449807
12500000 14.991126954343365
12510000 14.470869318291676
12520000 13.663689457261402
12530000 14.369766747518055
12540000 15.188894043835738
12550000 15.512421918691695
12560000 16.357108810211844
12570000 16.414292377201466
12580000 17.040836310274504

...

36190000 4.499768141958614
36200000 4.177289285965919
36210000 4.78486692590336
36220000 4.1401988796252605
36230000 4.593429449801094
36240000 4.464253368020667
36250000 4.721476195997436
36260000 4.227038156715307
36270000 3.5083327130819293
36280000 4.922630789623161
36290000 4.284442374109427
36300000 1.5168804054176144
36310000 0.9418823401308059
36320000 0.993815406270014
36330000 3.140102349998944
36340000 5.645389639977376
36350000 3.8246140977366303
36360000 2.874153215573715
36370000 3.4025503007465203
36380000 4.550483435113126
36390000 4.899642428150468
36400000 3.1298224379573463
36410000 4.186087370000495
36420000 4.009212283364428
36430000 4.375484423602979
36440000 4.689390868594203
36450000 2.560774167109754
36460000 1.5790533264848987
36470000 1.814948189331571
36480000 2.8196598499519694
36490000 4.916098507910609
36500000 4.928220223606779
36510000 4.78778782786517
36520000 4.763574930742542
36530000 4.540388726826224
36540000 4.640363181111646
36550000 4.2306868447

39190000 2.2336385056790116
39200000 3.705717561725842
39210000 2.430126787159476
39220000 2.4651848798437315
39230000 3.4346904217365584
39240000 2.4243659021014965
39250000 2.809352900611818
39260000 2.8676442943380014
39270000 3.8446729249786404
39280000 2.2056802353968026
39290000 3.2980617127943637
39300000 3.1601367978918615
39310000 2.448887864075065
39320000 3.3394889041143854
39330000 3.9025653775084685
39340000 2.5309326083010633
39350000 0.7835696463064088
39360000 1.0881769867086544
39370000 0.9518331997177005
39380000 3.777760514045861
39390000 3.046986339446836
39400000 3.0153063975357215
39410000 2.5641365436895622
39420000 2.9408697337495022
39430000 2.932990184635858
39440000 3.1342964226784704
39450000 2.9446424513129
39460000 3.204640974878192
39470000 3.3910984987055395
39480000 3.37486934107436
39490000 3.1244582661822635
39500000 3.208578515867227
39510000 3.2167975057259333
39520000 2.7044259903659817
39530000 2.5092276147787835
39540000 1.4349390295744802
395500

42160000 2.349347137949586
42170000 2.3626700236366456
42180000 1.8612164853181374
42190000 1.6290745470603307
42200000 1.4391204742342698
42210000 1.7102825014534462
42220000 1.7291587596476274
42230000 2.2991682678987315
42240000 0.9438421138807868
42250000 0.7169940956553817
42260000 0.6602280328937967
42270000 1.7337061515211
42280000 2.69065240419362
42290000 2.242742794544677
42300000 1.8161317492238418
42310000 2.3098561016431054
42320000 1.5833609650329485
42330000 1.9213516025134523
42340000 1.5631769798262118
42350000 1.7235207518326443
42360000 1.2935545635769103
42370000 0.9800863205217918
42380000 0.5877531940371924
42390000 0.9986845228349434
42400000 2.1680646788484257
42410000 1.4201952378361558
42420000 1.2739464823776618
42430000 1.8026327418240904
42440000 1.7596969343641533
42450000 1.8264559936183493
42460000 1.8046066437571644
42470000 1.9110007814657226
42480000 2.1449950502568487
42490000 2.32903544441549
42500000 1.9732167286608815
42510000 1.9975595222560034
4

45100000 0.8495990249886314
45110000 0.7340380189373759
45120000 0.9881096787158052
45130000 0.6188615272775888
45140000 0.31074058776948843
45150000 0.47161246138752166
45160000 0.5628418527578115
45170000 1.2966661598967113
45180000 0.9453716602814264
45190000 0.42540097156776985
45200000 0.4553912865008646
45210000 0.5132374565142526
45220000 0.9924867265046834
45230000 1.1582237825141084
45240000 0.7329714624115493
45250000 0.9435123694993257
45260000 1.1741539606370264
45270000 0.8400487921352254
45280000 0.8968838306595882
45290000 0.6254766362731722
45300000 0.7404046398797896
45310000 0.7688568087490797
45320000 0.6501653792976406
45330000 0.833812273167173
45340000 0.7558430525657535
45350000 0.653328747264372
45360000 0.8343081751048698
45370000 0.6378578412683805
45380000 1.120037299562441
45390000 0.6771835091586907
45400000 0.9146217123679519
45410000 0.7835155288155079
45420000 0.8735809391858312
45430000 0.6043723274793823
45440000 0.7517023824284209
45450000 0.124678829

47960000 0.005086834847033024
47970000 0.0044911673548751405
47980000 0.003797676568686963
47990000 0.003105024515926838
48000000 0.0022446406401793162
48010000 0.001779536492049694
48020000 0.0011045385602447724
48030000 0.0004566530363427268
best so far: 0
type: [1, 1, 83, 1, 1] 83
cases of this type: 6889
best so far: 0
type: [1, 83, 1, 1, 1] 83
cases of this type: 83
best so far: 0
type: [83, 1, 1, 1, 1] 83
cases of this type: 1
best so far: 0
all cases: 48037081
best: 0
